#### LLM chaining
  🔹 LLMChain
  
        SequentialChain
        RouterChain
        TransformChain
        RetrievalChain (v0.1) ✔️ Correct — now part of retrieval module
    
  🔹 NEW in LangChain 0.1+
  
        Runnable sequences
        Runnable mapping
        Parallel chains
        Streaming pipelines

Chains = connecting multiple components into a pipeline:

Prompt → LLM → Output Parser

Loader → Splitter → VectorStore → Retriever → LLM

Decision routing

Multi-step logic

Below are examples of ALL important chain types:

#### LLMChain — Basic Prompt → LLM

In [1]:
api_key = ''

In [2]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini",
                 api_key = api_key)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in one paragraph.",
)

parser = StrOutputParser()

chain = prompt | llm | parser

print(chain.invoke({"topic": "feature engineering"}))

### ✅ 3.2 SequentialChain — Multi-step pipeline

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Step 1: Generate ideas
idea_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate 3 creative ideas about {topic}."
)

# Step 2: Expand the best idea
expand_prompt = PromptTemplate(
    input_variables=["ideas"],
    template="Pick the best idea from this list and expand it into a paragraph:\n{ideas}"
)

# --- Build the pipeline ---

# Step 1 → produce "ideas"
step1 = idea_prompt | llm | parser

# Step 2 → consume "ideas" → produce final output
step2 = expand_prompt | llm | parser

# Complete chain
chain = (
    {"ideas": step1}  # Run step1 and store output in "ideas"
    | step2           # Pass "ideas" into step2
)

# Run the chain
result = chain.invoke({"topic": "AI in healthcare"})
print(result)



### ⭐ 3.3 RouterChain — Choose the right chain based on input

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

parser = StrOutputParser()

# Destination prompts
math_prompt = PromptTemplate.from_template("Solve this math problem: {input}")
joke_prompt = PromptTemplate.from_template("Tell a joke about: {input}")
define_prompt = PromptTemplate.from_template("Provide a definition of: {input}")

# Default fallback prompt
default_prompt = PromptTemplate.from_template("Answer this question normally: {input}")

# Router logic (replacement for MultiPromptChain)
router = RunnableBranch(
    # If input looks like math
    (lambda x: any(c.isdigit() for c in x["input"]), math_prompt | llm | parser),

    # If input mentions "joke"
    (lambda x: "joke" in x["input"].lower() or "funny" in x["input"].lower(),
        joke_prompt | llm | parser),

    # If input asks for definition
    (lambda x: "define" in x["input"].lower() or "what is" in x["input"].lower(),
        define_prompt | llm | parser),

    # Default fallback
    default_prompt | llm | parser
)

# Test
print(router.invoke({"input": "5 + 7"}))
print(router.invoke({"input": "Tell me something funny"}))
print(router.invoke({"input": "Explain transformers"}))

In [ ]:
### ⭐ TransformChain

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

#llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

# Step 1: Clean the text
clean_step = RunnableLambda(lambda x: {"text": x["text"].strip()})

# Step 2: Format into a prompt (REQUIRED!)
prompt = PromptTemplate.from_template("Answer this: {text}")

# Step 3: Final chain
chain = clean_step | prompt | llm | parser

print(chain.invoke({"text": "   Explain feature engineering   "}))

### 🔵 2. RetrievalChain — Auto RAG Pipeline

        A RetrievalChain combines:

            Retriever

            Prompt

            LLM

            Answer

In [ ]:
#pip install -U langchain langchain-community langchain-core langchain-openai langchain-text-splitters

In [3]:
import langchain
print(langchain.__version__)

1.1.2


In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [5]:


# --------------------------
# Sample documents
# --------------------------
docs = [
    "LangChain is a framework for building LLM applications.",
    "FAISS is used for fast vector similarity search.",
    "RAG improves LLM accuracy using document retrieval."
]

# --------------------------
# Split documents
# --------------------------
splitter = RecursiveCharacterTextSplitter(chunk_size=200)
documents = splitter.create_documents(docs)

# --------------------------
# Create vector store + retriever
# --------------------------
embeddings = OpenAIEmbeddings(api_key=api_key)
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# --------------------------
# Build the prompt template
# --------------------------
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using ONLY the context."),
    ("human", "Context:\n{context}\n\nQuestion:\n{question}")
])

# --------------------------
# LLM
# --------------------------
llm = ChatOpenAI(model = "gpt-4o-mini",
                 api_key = api_key)

# --------------------------
# Build RAG pipeline (LCEL)
# --------------------------
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# --------------------------
# Ask a question
# --------------------------
response = rag_chain.invoke("What is FAISS used for?")
print(response)



content='FAISS is used for fast vector similarity search.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 163, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6318584bd8', 'id': 'chatcmpl-CkfIYaiQR18RgEmmQSVUfM6KDKDp4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b0060-574a-7bb3-be9f-1b29458deb5e-0' usage_metadata={'input_tokens': 163, 'output_tokens': 10, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
